### Spotify

Rangsiman Ketkaew

In [2]:
import spotipy
# import spotipy.util as util
from lyricsgenius import Genius
import json
import pandas as pd
import numpy as np

# Import token for getting song's info from Spotify,
# and also token for Genius website
from my_token import *

#### Spotify Scope

In [163]:
scope = 'user-read-currently-playing'
# scope = 'user-modify-playback-state'
# scope = 'user-read-recently-played'

# To connect succesfully you need to provide your own Spotify Credentials
# You can do this signing up in https://developer.spotify.com/ and creating a new app.
token = spotipy.util.prompt_for_user_token(
    USERNAME, scope, client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET, redirect_uri=SPOTIPY_REDIRECT_URI)

sp = spotipy.Spotify(auth=token)

In [164]:
# method currently playing return an actual song on Spotify
current_song = sp.currently_playing()
current_song['item']['id']

In [154]:
# Skip to the next track
# Don't fotget to update a scope with 'user-modify-playback-state'
sp.next_track()

### Functions to get info

In [170]:
def get_features(track_id):
    features_results = sp.audio_features([track_id])
    json_features = json.dumps(features_results)
    features_data = json.loads(json_features)

    # Convert features dictionary to a list
    features_list = list(features_data[0].values())

    return features_list

In [157]:
def get_lyrics(song_name, artist_name):
    genius = Genius(token, timeout=100)
    genius.verbose = False
    genius.remove_section_headers = True
    song = genius.search_song(song_name, artist_name)
    try:
        return song.lyrics.replace('\n'," ")
    except:
        return None

#### Check if the song is skipped

In [159]:
import time

def calc_end_time():
    start_time = time.time()
    length_of_the_song = current_song['item']['duration_ms'] / 1000 
    endtime = start_time + length_of_the_song
    return endtime

In [ ]:
all_non_skipted = []
all_skipted = [] 

### If song is skipped its puts features into all_skipted, if not all_non_skipted

current_song = sp.currently_playing()
name_of_song = current_song['item']['name']
endtime = calc_end_time()

while True:
    if time.time() < endtime:
        current_song = sp.currently_playing()
        new_name = current_song['item']['name']
        old_track_id = current_song['item']['id']
        if  new_name ==  name_of_song:
            pass
        else:
            track_id = current_song['item']['id']
            features= get_features(track_id)
            all_skipted.append(features)
    elif time.time() > endtime:
        features_old= get_features(old_track_id)
        all_non_skipted.append(features_old)

    name_of_song = new_name
    time.sleep(5)
    # print(all_skipted)

#### Get last 50 recentlyed played songs

In [5]:
user = sp.user("nutty_chem")
# you need to use scope: user-read-recently-played
last_50_songs = sp.user_playlists(user['id'], limit=50)
# last_50_songs

In [6]:
last_50 = sp.current_user_recently_played()
print(last_50['items'][49]['track']['name'])
print(last_50['items'][49]['track']['artists'][0]['name'])

7 Years
Lukas Graham


In [7]:
last_50['items'][49]['track']['id']

'23ihGIrYOSzNNGJ6STAxI4'

In [10]:
# Audio features
feature_names = [
  "danceability",
  "energy",
  "key",
  "loudness",
  "mode",
  "speechiness",
  "acousticness",
  "instrumentalness",
  "liveness",
  "valence",
  "tempo",
  # "type",
  # "id",
  # "uri",
  # "track_href",
  # "analysis_url",
  # "duration_ms",
  # "time_signature"
]

In [11]:
# Write csv data file
import csv
data_file = open('data.csv','w')
writer = csv.writer(data_file)
writer.writerow(['track_id', 'track_name', 'first_artist'] + feature_names + ['neg', 'neu', 'pos', 'compound'])

158

In [12]:
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sentiment_analyzer = SIA()

repeat_query = True
offset_n = 0
for i in range(2):
    # Query Spotify API
    if i > 0:
        print('Repeating query')
        offset_n += 100

    # Write rows
    for i in range(50):
        track_id = last_50['items'][i]['track']['id']
        print(track_id)
        
        # date_added = track['added_at']
        track_name = last_50['items'][i]['track']['name']
        first_artist = last_50['items'][i]['track']['artists'][0]['name']

        # Track features
        features = get_features(track_id)
    
        # Try to get lyrics, if available
        lyrics = get_lyrics(track_name, first_artist)

        # Sentiment Analysis
        neg = None
        neu = None
        pos = None
        compound = None
        if lyrics:
            snt = sentiment_analyzer.polarity_scores(lyrics)
            neg = snt['neg']
            neu = snt['neu']
            pos = snt['pos']
            compound = snt['compound']

        writer.writerow([track_id, track_name, first_artist] + features[:-7] + [neg, neu, pos, compound])

print('Done querying')
data_file.close()


In [1]:
import pandas as pd 
pd.read_csv("../data/player_data/skipped.csv")

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,skip_state,date_saved,skip_probability
0,0,0.744,0.578,8,-10.574,0,0.0286,0.31900,0.133,0.104,...,audio_features,2rXC8O7kaYw7FjStybL2rz,spotify:track:2rXC8O7kaYw7FjStybL2rz,https://api.spotify.com/v1/tracks/2rXC8O7kaYw7...,https://api.spotify.com/v1/audio-analysis/2rXC...,202158,4,SKIPPEP,2022-10-23 12:03:19.042178,0.0
1,1,0.378,0.122,2,-19.785,1,0.0360,0.00697,0.958,0.125,...,audio_features,0ttF21hZCPw2merE3GsHcP,spotify:track:0ttF21hZCPw2merE3GsHcP,https://api.spotify.com/v1/tracks/0ttF21hZCPw2...,https://api.spotify.com/v1/audio-analysis/0ttF...,29827,4,SKIPPEP,2022-10-23 12:03:51.343571,0.0
2,2,0.740,0.266,1,-17.222,1,0.9470,0.98600,0.000,0.508,...,audio_features,0EgkVzDidmq4MP57jZFeik,spotify:track:0EgkVzDidmq4MP57jZFeik,https://api.spotify.com/v1/tracks/0EgkVzDidmq4...,https://api.spotify.com/v1/audio-analysis/0Egk...,22747,3,NaN,NaN,NaN
